In [ ]:
import numpy as np
import scipy.stats
from scipy.stats import multivariate_normal, norm
from matplotlib import pyplot as plt


######EXAMPLES FROM DOC##################
# define a uniform distribution
# on [3, 3] and a normal distribution (restricted to the range [-3,3]) by the following two definitions
distrib0 = scipy.stats.truncnorm(-3,3,loc=0,scale=1)
distrib1 = scipy.stats.uniform(loc=-3,scale=6)

# We can perform one-dimensional Monte Carlo integration with the following code snippet:
def integrate_me(f, distrib, npts=100):
    x = distrib.rvs(npts)
    ps = distrib.pdf(x)
    f = f(x)
    mu = np.mean(f/ps)
    err = np.std(f/ps)/np.sqrt(npts)
    return mu,err


class MyTwoDUniform(object):
    def __init__(self, bounds=None):
        self.bounds =np.array(bounds)
    def rvs(self,npts):
        my_out = np.empty( (len(self.bounds),npts))
        for dim in np.arange(len(self.bounds)):
            my_out[dim] = np.random.uniform(low=self.bounds[dim][0], high=self.bounds[dim][1], size=npts)
        return my_out.T
    def pdf(self,x):
        V = np.prod([self.bounds[:,1]- self.bounds[:,0]])
        return np.ones(x.shape[0])/V


#Main Stuff
#EXAMPLES FROM DOC
# Example of integration
print("DOC EXAMPLES")
my2d = MyTwoDUniform(bounds=[[-3,3],[-3,3]])
mu = np.array([1,-0.5])
cov = np.array([[ 1. , -0.1 ], [-0.1 , 0.05]])
def f(x1, x2):
    x = np.array([x1, x2]).T
    return multivariate_normal.pdf(x, mu, cov)
print(integrate_me(lambda x: f(*(x.T)), my2d))
print("________________________________")
#End of examples

#Getting Started
print("Integral Estimates: (Uniform)")
EVALS = 10**3
distribQ10 = scipy.stats.uniform(loc=-10,scale=20)
distribQ1 = scipy.stats.uniform(loc=-1,scale=2)

def xeq1(x):
    return 1
print(f"Q1 [-10,10] f(x)=1  :  {integrate_me(xeq1, distribQ10, EVALS)} \t Actual integral(-10, 10)(x)dx = 20")


print(f"Q2 [-1,1] f(x)= eval legendre(l,x)^2 for[1,6]:")
for l in range(0,7):
    def plsquared(x):
        out = scipy.special.eval_legendre(l,x)
        return (out*out)
    print(f"\t l={l}   :   {integrate_me(plsquared, distribQ1, EVALS)}")

# standard normal PDF with mean of 0 and variance 1
e = 2.718281828459045
pi = 3.1415926535897932384626

def f(x):
    mu = 0
    sd = 1
    bottom = 2*pi
    bottom = bottom**.5
    bottom = bottom*sd
    exp = x-mu
    exp = exp/sd
    exp = exp**2
    exp = exp/-2
    top = e**exp
    return top/ bottom
#split up into parts
Half = scipy.stats.uniform(-10,10)
mp = integrate_me(f, Half, EVALS)
mu = 2*mp[0]
err = 2*mp[1]
print(f"Q3 [-10,10] f(x)=standard normal PDF with mean of 0 and variance 1  :  {(mu, err)}")


def p1(x):
    mu = -3
    sd = 1
    bottom = 2*pi
    bottom = bottom**.5
    bottom = bottom*sd
    exp = x-mu
    exp = exp/sd
    exp = exp**2
    exp = exp/-2
    top = e**exp
    return top/ bottom

def p2(x):
    mu = 3
    sd = 3
    bottom = 2*pi
    bottom = bottom**.5
    bottom = bottom*sd
    exp = x-mu
    exp = exp/sd
    exp = exp**2
    exp = exp/-2
    top = e**exp
    return top/ bottom

# distribQ10 = scipy.stats.truncnorm(-10,10,loc=0,scale=1)
HalfP1 = scipy.stats.uniform(-10,7)
HalfP2 = scipy.stats.uniform(-10,13)
a = integrate_me(p1, HalfP1, EVALS)
b = integrate_me(p2, HalfP2, EVALS)
mu = 1.4 * a[0] + .6* b[0]
err = 1.4 *a[1] + .6* b[1]
print(f"Q4 [-10,10] f(x)=0.7p1(x) + 0.3p2(x)  :  {(mu , err)}")



print()
print("_________________________________________")
print("Integral Estimates 1: (Truncated Normal)")
EVALS = 10**3
EVALS = 10**3
distribQ10 = scipy.stats.truncnorm(-10,10,loc=0,scale=1)
distribQ1 = scipy.stats.truncnorm(-1,1,loc=0,scale=1)

def xeq1(x):
    return 1
print(f"Q1 [-10,10] f(x)=1  :  {integrate_me(xeq1, distribQ10, EVALS)} \t Actual integral(-10, 10)(x)dx = 20")


# standard normal PDF with mean of 0 and variance 1
e = 2.718281828459045
pi = 3.1415926535897932384626

def f(x):
    mu = 0
    sd = 1
    bottom = 2*pi
    bottom = bottom**.5
    bottom = bottom*sd
    exp = x-mu
    exp = exp/sd
    exp = exp**2
    exp = exp/-2
    top = e**exp
    return top/ bottom
#split up into parts
Half = scipy.stats.truncnorm(-10,0,loc=0,scale=1)
mp = integrate_me(f, Half, EVALS)
mu = 2*mp[0]
err = 2*mp[1]
print(f"Q3 [-10,10] f(x)=standard normal PDF with mean of 0 and variance 1  :  {(mu, err)}")


def p1(x):
    mu = -3
    sd = 1
    bottom = 2*pi
    bottom = bottom**.5
    bottom = bottom*sd
    exp = x-mu
    exp = exp/sd
    exp = exp**2
    exp = exp/-2
    top = e**exp
    return top/ bottom

def p2(x):
    mu = 3
    sd = 3
    bottom = 2*pi
    bottom = bottom**.5
    bottom = bottom*sd
    exp = x-mu
    exp = exp/sd
    exp = exp**2
    exp = exp/-2
    top = e**exp
    return .3*top/ bottom

# distribQ10 = scipy.stats.truncnorm(-10,10,loc=0,scale=1)
P1 = scipy.stats.truncnorm(-10,-3,loc=0,scale=1)
P2 = scipy.stats.truncnorm(-10,3,loc=0,scale=1)
resP1 = integrate_me(p1, P1, EVALS)
resP2 = integrate_me(p2, P2, EVALS)
mu = 1.4*resP1[0] + .6*resP2[0]
err = 1.4*resP1[1] + .6*resP2[1]
print(f"Q4 [-10,10] f(x)=0.7p1(x) + 0.3p2(x)  :  {(mu , err)}")







DOC EXAMPLES
(0.6203889184561546, 0.248004748472974)
________________________________
Integral Estimates: (Uniform)
Q1 [-10,10] f(x)=1  :  (20.0, 0.0) 	 Actual integral(-10, 10)(x)dx = 20
Q2 [-1,1] f(x)= eval legendre(l,x)^2 for[1,6]:
	 l=0   :   (2.0, 0.0)
	 l=1   :   (0.6556930735369676, 0.01863067189303344)
	 l=2   :   (0.4119320528326067, 0.013519650143850145)
	 l=3   :   (0.3012630458824533, 0.011379981887105931)
	 l=4   :   (0.23740922867943892, 0.009758622985084734)
	 l=5   :   (0.17341010087052502, 0.006888726501641161)
	 l=6   :   (0.1595716336317566, 0.006205993651957593)
Q3 [-10,10] f(x)=standard normal PDF with mean of 0 and variance 1  :  (1.1115724763354, 0.07120108790768175)
Q4 [-10,10] f(x)=0.7p1(x) + 0.3p2(x)  :  (0.9835414311908914, 0.04902925034722637)

_________________________________________
Integral Estimates 1: (Truncated Normal)
Q1 [-10,10] f(x)=1  :  (8.139332165578187, 2.0574828261254803) 	 Actual integral(-10, 10)(x)dx = 20
Q3 [-10,10] f(x)=standard normal P

In [ ]:
print()
print("_________________________________________")
print("Rosenbrock Function:")
BOUNDS = [-5,5]
HALFBOUNDS = [-5,1]
# UNIFORMDIS = scipy.stats.truncnorm(-5,-0,loc=0,scale=1)

class Rosenbrook(object):
    def __init__(self, bounds=np.array([[-5,5],[-5,5]])):
        self.bounds =np.array(bounds)
    def rvs(self,npts):
        my_out = np.empty( (len(self.bounds),npts))
        for dim in np.arange(len(self.bounds)):
            my_out[dim] = np.random.uniform(low=self.bounds[dim][0], high=self.bounds[dim][1], size=npts)
        return my_out.T
    def pdf(self,x):
        V = np.prod([self.bounds[:,1]- self.bounds[:,0]])
        return np.ones(x.shape[0])/V

def ln_f(x1, x2):
  minus_lnL = np.array(np.power((1.-x1), 2) + 100.* np.power((x2-x1**2),2),dtype=float)
  return - minus_lnL


def g(x1, x2):
  return np.exp(ln_f(x1,x2))

def integrate_me_rosen(f, distrib, npts=100):
    x = distrib.rvs(npts)
    ps = distrib.pdf(x)
    f = f(x)
    mu = np.mean(f/ps)
    err = np.std(f/ps)/np.sqrt(npts)
    return mu,err


Rosen1 = Rosenbrook()
result = integrate_me_rosen(lambda x: g(*(x.T)), Rosen1, 10**7)
print(result)
print()
print("Using Half Bounds:")
Rosen2 = Rosenbrook(bounds=[HALFBOUNDS, HALFBOUNDS])
result = integrate_me_rosen(lambda x: g(*(x.T)), Rosen2, 10**7)
result = (result[0]*2, 2*result[1])
print(result)

# Rosen2 = Rosenbrook(bounds=[HALFBOUNDS,HALFBOUNDS])
# result = integrate_me(lambda x: f(*(x.T)), Rosen1, 10**5)
# print((2*result[0], result[1]))



_________________________________________
Rosenbrock Function:
(0.3020281137285295, 0.001247352268020254)

Using Half Bounds:
(0.30685902544424765, 0.0010488231121489155)
